In [1]:
from pyspark.sql import SparkSession

spark = SparkSession.builder \
    .appName("Running SQL Queries for IMDb") \
    .getOrCreate()

Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
23/10/25 17:58:43 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [2]:
df1 = spark.read.csv("ImdbName.csv", header = True)
df1.createOrReplaceTempView("name_basics") # Table name

df2 = spark.read.csv("ImdbTitleBasics.csv", header = True)
df2.createOrReplaceTempView("title_basics")

df3 = spark.read.csv("ImdbTitleCrew.csv", header = True)
df3.createOrReplaceTempView("title_crew")

df4 = spark.read.csv("ImdbTitlePrincipals.csv", header = True)
df4.createOrReplaceTempView("title_principals")

df5 = spark.read.csv("ImdbTitleRatings.csv", header = True)
df5.createOrReplaceTempView("title_ratings")

In [6]:
df2.show()

+---------+---------+--------------------+--------------------+-------+---------+-------+--------------+--------------------+
|   tconst|titleType|        primaryTitle|       originalTitle|isAdult|startYear|endYear|runtimeMinutes|              genres|
+---------+---------+--------------------+--------------------+-------+---------+-------+--------------+--------------------+
|tt0000001|    short|          Carmencita|          Carmencita|      0|     1894|     \N|             1|   Documentary,Short|
|tt0000002|    short|Le clown et ses c...|Le clown et ses c...|      0|     1892|     \N|             5|     Animation,Short|
|tt0000003|    short|      Pauvre Pierrot|      Pauvre Pierrot|      0|     1892|     \N|             4|Animation,Comedy,...|
|tt0000004|    short|         Un bon bock|         Un bon bock|      0|     1892|     \N|            12|     Animation,Short|
|tt0000005|    short|    Blacksmith Scene|    Blacksmith Scene|      0|     1893|     \N|             1|        Comedy

In [3]:
import json

with open('./predicted_results.jsonl', 'r') as json_file:
    json_list = [json.loads(line) for line in json_file]

ground_truth_dict = {}
for json_obj in json_list:
    ground_truth_dict[json_obj['id']] = json_obj['groundtruth']

In [4]:
ground_truth_dict

{20001: ['Police Academy  (1984)',
  'Police Academy 2: Their First Assignment (1985)'],
 20047: ['The Black Panthers: Vanguard of the Revolution (2015)',
  'It  (2017)',
  'Avengers: Infinity War (2018)'],
 20048: ['Ant-Man  (2015)', 'Cabin by the Lake (2000)'],
 20051: ['X-Men: The Last Stand (2006)'],
 20052: ['It  (2017)', 'Avengers: Infinity War (2018)'],
 20053: ['It  (2017)', 'The Last Exorcism (2010)'],
 20054: ['The Mask (1988)'],
 20055: ['The Exorcist  (1973)', 'Avengers: Infinity War (2018)'],
 20057: ['Spider-Man: Homecoming (2017)'],
 20058: ['The Big Lebowski'],
 20059: ['House of 1000 Corpses (2003)', 'Blazing Saddles (1974)'],
 20083: ['The Sixth Battalion (1998)'],
 20095: ['The Conjuring (2013)'],
 20098: ['Billy Madison (1995)'],
 20121: ['Annihilation  (2018)'],
 20133: ['The Sixth Sense (1999)'],
 20148: ['Happy Death Day  (2017)', 'Creed  (2015)'],
 20185: ['Donnie Darko (2001)',
  'A Quiet Place (2018)',
  "A Nightmare on Elm Street 2: Freddy's Revenge (1985)",


In [ ]:
import glob
from tqdm import tqdm
import numpy as np
non_executable = []

hit_rate = [] # number of hitted movie / total number of ground-truth movies
recall_rate = [] # total number of ground-truth movies / generated recommendations

for filepath in tqdm(glob.iglob('query_results/*.sql')):

    try:
        with open(filepath) as sql_file:
            sql_query = sql_file.read()
        result = spark.sql(sql_query).select("primaryTitle").rdd.flatMap(lambda x: x).collect()
        if len(result) > 0:
            num = filepath.split('/')[1].split('.')[0]
            ground_truth = ground_truth_dict[int(num)]
            # print(ground_truth)
            recall_rate.append(len(ground_truth)/len(result))
            print("recall rate = ", np.mean(np.array(recall_rate)))
            hit = 0
            for i in ground_truth:
                i = i.split('(')[0].strip()
                # print(i)
                if i in result:
                    hit += 1
            hit_rate.append(hit/len(ground_truth))
            print("hit rate = ",np.mean(np.array(hit_rate)))
            
            
    except:
        non_executable.append(filepath)

23/10/25 17:58:55 WARN GarbageCollectionMetrics: To enable non-built-in garbage collector(s) List(G1 Concurrent GC), users should configure it(them) to spark.eventLog.gcMetrics.youngGenerationGarbageCollectors or spark.eventLog.gcMetrics.oldGenerationGarbageCollectors
94it [00:02, 38.58it/s]                                                         

recall rate =  0.0033222591362126247
hit rate =  0.0


392it [00:04, 137.07it/s]

recall rate =  0.07308970099667773
hit rate =  0.0


456it [00:05, 198.09it/s]

...

4719it [01:25, 59.80it/s]                                                       

recall rate =  0.22459248325398204
hit rate =  0.24645390070921988


4773it [01:25, 88.05it/s]

recall rate =  0.22008148393887886
hit rate =  0.24131944444444442


[Stage 187:>                                                      (0 + 10) / 10]
Exception: java.lang.OutOfMemoryError thrown from the UncaughtExceptionHandler in thread "RemoteBlock-temp-file-clean-thread"
5067it [01:40, 212.93it/s]
Exception: java.lang.OutOfMemoryError thrown from the UncaughtExceptionHandler in thread "Spark Context Cleaner"

Exception: java.lang.OutOfMemoryError thrown from the UncaughtExceptionHandler in thread "stdout writer for python3"
Exception in thread "stdout writer for python3" Exception in thread "stdout writer for python3" Exception in thread "stdout writer for python3" Exception in thread "refresh progress" java.lang.OutOfMemoryError: Java heap space
java.lang.OutOfMemoryError: Java heap space
ERROR:root:KeyboardInterrupt while sending command.
Traceback (most recent call last):
  File "/Library/Frameworks/Python.framework/Versions/3.10/lib/python3.10/site-packages/py4j/java_gateway.py", line 1038, in send_command
    response = connection.send_command(